In [1]:
import threading

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
spark

In [4]:
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, array, col, lit
from pyspark.sql.types import StringType

In [5]:
from pyspark.ml import PipelineModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

globals()['models_loaded'] = False
globals()['my_model'] = None

# Create evaluator
evaluatorMulti = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

def process(time, rdd):
    if rdd.isEmpty():
        return
    
    print("========= %s =========" % str(time))
    
    # Convert to data frame
    df = spark.read.json(rdd)
    df.show()
    
    # Load in the model if not yet loaded:
    if not globals()['models_loaded']:
        # load in your models here
        globals()['my_model'] =  PipelineModel.load("pipeline_model")
        globals()['models_loaded'] = True
        
    # And then predict using the loaded model: 
    df_result = globals()['my_model'].transform(df)
    
    # Evaluation 
    predictionAndTarget = df_result.select("label", "prediction")
    acc = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "accuracy"})
    f1 = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "f1"})
    f1_label_0 = evaluatorMulti.evaluate(predictionAndTarget, 
                                {evaluatorMulti.metricName: "fMeasureByLabel", evaluatorMulti.metricLabel:0.0})
    f1_label_1 = evaluatorMulti.evaluate(predictionAndTarget, 
                                {evaluatorMulti.metricName: "fMeasureByLabel", evaluatorMulti.metricLabel:1.0})
    f1_label_2 = evaluatorMulti.evaluate(predictionAndTarget, 
                                {evaluatorMulti.metricName: "fMeasureByLabel", evaluatorMulti.metricLabel:2.0})
    weightedPrecision = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "weightedPrecision"})
    weightedRecall = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "weightedRecall"})
    
    print(f"F1 Score for #loltyler1: {f1_label_0}")
    print(f"F1 Score for #easportsfifa: {f1_label_1}")
    print(f"F1 Score for #trainwreckstv: {f1_label_2}\n")
    print(f"Accuracy: {acc}")
    print(f"F1 Score: {f1}")
    print(f"Weighted Precision: {weightedPrecision}")
    print(f"Weighted Recall: {weightedRecall}")
    df_result.select("channel", "message", "label", "prediction").show()
    

In [6]:
ssc = StreamingContext(sc, 10)

In [7]:
lines = ssc.socketTextStream("localhost", 8080)
lines.foreachRDD(process)

In [9]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

22/05/27 18:12:08 WARN StreamingContext: StreamingContext has already been started
22/05/27 18:12:09 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/05/27 18:12:09 WARN BlockManager: Block input-0-1653667929400 replicated to only 0 peer(s) instead of 1 peers
22/05/27 18:12:10 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/05/27 18:12:10 WARN BlockManager: Block input-0-1653667930400 replicated to only 0 peer(s) instead of 1 peers


========= 2022-05-27 18:12:10 =========


22/05/27 18:12:11 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/05/27 18:12:11 WARN BlockManager: Block input-0-1653667931400 replicated to only 0 peer(s) instead of 1 peers
22/05/27 18:12:12 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/05/27 18:12:12 WARN BlockManager: Block input-0-1653667932400 replicated to only 0 peer(s) instead of 1 peers
22/05/27 18:12:13 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/05/27 18:12:13 WARN BlockManager: Block input-0-1653667933400 replicated to only 0 peer(s) instead of 1 peers


+--------------+--------------------+--------------------+-----------------+
|       channel|            datetime|             message|         username|
+--------------+--------------------+--------------------+-----------------+
|#trainwreckstv|2022-05-27T16:12:...|https://twitter.c...|       streamlabs|
| #easportsfifa|2022-05-27T16:12:...|          Serie A <3|         kishod25|
|    #loltyler1|2022-05-27T16:12:...|               !opgg|         zorkio92|
| #easportsfifa|2022-05-27T16:12:...|argentineans dont...|        dezorisfc|
|    #loltyler1|2022-05-27T16:12:...|@zorkio92 https:/...|         fossabot|
|#trainwreckstv|2022-05-27T16:12:...|this game blows  ...|       drewzus520|
|    #loltyler1|2022-05-27T16:12:...|           PotFriend|     rand0mguy360|
|    #loltyler1|2022-05-27T16:12:...|           PartyTime|falloutpipboy3000|
|    #loltyler1|2022-05-27T16:12:...|I want to squeeze...|         gaoferry|
|    #loltyler1|2022-05-27T16:12:...|PotFriend PotFrie...|      tructrucker|

22/05/27 18:12:14 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/05/27 18:12:14 WARN BlockManager: Block input-0-1653667934400 replicated to only 0 peer(s) instead of 1 peers
22/05/27 18:12:15 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/05/27 18:12:15 WARN BlockManager: Block input-0-1653667935400 replicated to only 0 peer(s) instead of 1 peers
22/05/27 18:12:16 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/05/27 18:12:16 WARN BlockManager: Block input-0-1653667936400 replicated to only 0 peer(s) instead of 1 peers
22/05/27 18:12:17 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/05/27 18:12:17 WARN BlockManager: Block input-0-1653667937400 replicated to only 0 peer(s) instead of 1 peers
22/05/27 18:12:18 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/05/27 18:12:18 WARN BlockManager: Block input-0-1653667938400 replicated to

...

22/05/27 18:12:41 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/05/27 18:12:41 WARN BlockManager: Block input-0-1653667961400 replicated to only 0 peer(s) instead of 1 peers
22/05/27 18:12:41 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/05/27 18:12:41 WARN BlockManager: Block input-0-1653667961600 replicated to only 0 peer(s) instead of 1 peers
22/05/27 18:12:42 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/05/27 18:12:42 WARN BlockManager: Block input-0-1653667962400 replicated to only 0 peer(s) instead of 1 peers
22/05/27 18:12:42 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/05/27 18:12:42 WARN BlockManager: Block input-0-1653667962600 replicated to only 0 peer(s) instead of 1 peers
22/05/27 18:12:43 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/05/27 18:12:43 WARN BlockManager: Block input-0-1653667963600 replicated to

========= 2022-05-27 18:12:50 =========
+--------------+--------------------+--------------------+-------------------+
|       channel|            datetime|             message|           username|
+--------------+--------------------+--------------------+-------------------+
|    #loltyler1|2022-05-27T16:12:...|               !rank|        jimboslic33|
| #easportsfifa|2022-05-27T16:12:...|LO DIJO BIEN CLAR...|            sebam75|
|    #loltyler1|2022-05-27T16:12:...|        time to roam|          falls2010|
|    #loltyler1|2022-05-27T16:12:...|  Shot like dogs LUL|            grimeto|
|#trainwreckstv|2022-05-27T16:12:...|What is the max w...|     thedarknighttt|
|    #loltyler1|2022-05-27T16:12:...|lmao what was tha...|        bunnyzqueen|
|    #loltyler1|2022-05-27T16:12:...|               sez u|pepperspraylaughing|
|    #loltyler1|2022-05-27T16:12:...|            GET GOOD|          offens1v3|
| #easportsfifa|2022-05-27T16:12:...|        eafifaLetsGo|           josetej1|
|    #loltyl

22/05/27 18:12:50 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/05/27 18:12:50 WARN BlockManager: Block input-0-1653667970600 replicated to only 0 peer(s) instead of 1 peers


F1 Score for #loltyler1: 0.816326530612245
F1 Score for #easportsfifa: 0.6153846153846153
F1 Score for #trainwreckstv: 0.5333333333333333

Accuracy: 0.7111111111111111
F1 Score: 0.7106087563230421
Weighted Precision: 0.7266666666666667
Weighted Recall: 0.711111111111111
+--------------+--------------------+-----+----------+
|       channel|             message|label|prediction|
+--------------+--------------------+-----+----------+
|    #loltyler1|               !rank|  0.0|       0.0|
| #easportsfifa|LO DIJO BIEN CLAR...|  1.0|       2.0|
|    #loltyler1|        time to roam|  0.0|       0.0|
|    #loltyler1|  Shot like dogs LUL|  0.0|       0.0|
|#trainwreckstv|What is the max w...|  2.0|       1.0|
|    #loltyler1|lmao what was tha...|  0.0|       0.0|
|    #loltyler1|               sez u|  0.0|       0.0|
|    #loltyler1|            GET GOOD|  0.0|       1.0|
| #easportsfifa|        eafifaLetsGo|  1.0|       1.0|
|    #loltyler1|        JESUS CHRIST|  0.0|       0.0|
|    #loltyler

22/05/27 18:12:51 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/05/27 18:12:51 WARN BlockManager: Block input-0-1653667971600 replicated to only 0 peer(s) instead of 1 peers
22/05/27 18:12:52 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/05/27 18:12:52 WARN BlockManager: Block input-0-1653667972600 replicated to only 0 peer(s) instead of 1 peers
22/05/27 18:12:53 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/05/27 18:12:53 WARN BlockManager: Block input-0-1653667973600 replicated to only 0 peer(s) instead of 1 peers
22/05/27 18:12:54 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/05/27 18:12:54 WARN BlockManager: Block input-0-1653667974600 replicated to only 0 peer(s) instead of 1 peers
22/05/27 18:12:55 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/05/27 18:12:55 WARN BlockManager: Block input-0-1653667975600 replicated to

In [11]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----


22/05/27 18:14:08 WARN StreamingContext: StreamingContext has already been stopped
